In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

is_kaggle = "KAGGLE_WORKING_DIR" in os.environ or "/kaggle" in os.getcwd()
print("Running on Kaggle:", is_kaggle)

Running on Kaggle: False


This is my attempt to reproduce the basic gradient descent used in fast.ai's ML for coder's course in Python. It's originally done in an excel spreadsheet. I don't have an excel license and the "Solver" functionality in excel is not available in numbers so I'm going to try and attempt it in Python with minimal use of typical ML frameworks

## Load Data set

In [6]:
if is_kaggle:
    data_path = "/kaggle/input/titanic/"
else:
    data_path = os.getcwd() + "/"
    
training_dataframe = pd.read_csv(data_path + "train.csv")
serving_df = pd.read_csv(data_path + "test.csv")

training_dataframe.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Declan/Projects/fastai-projects/titanictrain.csv'

## Prepare Data set
### Data Removal
First we'll remove the columns that won't be useful

In [ ]:
def remove_irrelevant_data(old_df: pd.DataFrame) -> pd.DataFrame:
    new_df = old_df.copy()
    columns_to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]
    return new_df.drop(columns=columns_to_drop)
    
training_dataframe = remove_irrelevant_data(training_dataframe)
training_dataframe.head(10)

There are also some rows with empty values we should remove

In [ ]:
def remove_na_values(old_df: pd.DataFrame) -> pd.DataFrame:
    cleaned_df = old_df.copy()
    cleaned_df = cleaned_df.dropna()
    removed_row_count = old_df.shape[0] - cleaned_df.shape[0]
    print(f"{removed_row_count} entries were removed, {cleaned_df.shape[0]} entries remain")
    return cleaned_df

training_dataframe = remove_na_values(training_dataframe)

### Converting Category Data to Binary Categorical Values
Sex, the Passenger class and Embarking city are not measurable attributes so we should convert them to Boolean numbers that can be used as co-efficients

In [ ]:
def convert_ticket_class_to_binary_values(original_df: pd.DataFrame) -> pd.DataFrame:
    new_df = original_df.copy()
    new_df["FirstClass"] = new_df["Pclass"].apply(lambda x: binary_equal_to_value(x,1))
    new_df["SecondClass"] = new_df["Pclass"].apply(lambda x: binary_equal_to_value(x,2))
    new_df.drop("Pclass", axis=1, inplace=True)
    return new_df
    

def binary_equal_to_value(number, compare_value):
    if (number == compare_value):
        return 1
    return 0

training_dataframe = convert_ticket_class_to_binary_values(training_dataframe)
training_dataframe.head(10)

In [ ]:
def convert_embarkation_port_to_binary_values(old_df: pd.DataFrame) -> pd.DataFrame:
    new_df = old_df.copy()
    new_df["Cherbourg_Departure"] = old_df["Embarked"].apply(lambda x: binary_equal_to_value(x, 'C'))
    new_df["Queenstown_Departure"] = old_df["Embarked"].apply(lambda x: binary_equal_to_value(x, 'Q'))
    new_df.drop("Embarked", axis=1, inplace=True)
    return new_df
    

training_dataframe = convert_embarkation_port_to_binary_values(training_dataframe)
training_dataframe.head(10)

In [ ]:
def convert_sex_to_binary_value(old_df: pd.DataFrame) -> pd.DataFrame:
    new_df = old_df.copy()
    new_df["Sex"] = old_df["Sex"].apply(lambda x: binary_equal_to_value(x, "male"))
    return new_df

training_dataframe = convert_sex_to_binary_value(training_dataframe)
training_dataframe.head(10)

### Converting numbers to fractional values
#### Age
Larger numbers would have too great an impact on our calculations so we can normalize them by dividing them by their max value them so they're between 0 and 1

In [ ]:
def convert_numeric_column_to_decimal(old_df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    new_df = old_df.copy()
    max_numeric_value = old_df[column_name].max()
    new_df[column_name] = old_df[column_name].apply(lambda x: x/max_numeric_value)
    return new_df
    
training_dataframe = convert_numeric_column_to_decimal(training_dataframe, "Age")
training_dataframe.head(10)

#### Fare
The `Fare` column has lots of small values with the occasional very large value. Uniform normalization using the max value isn't ideal when we're dealing with lots of small values with occasional very large values as the variation between the lower numbers will be lost. To normalize the values we can use a log function (log10 here) to bring the numbers down to reasonable ranges. We must use `log10(x+1)` to avoid 0 values as `log10(0)` would give us infinity.

In [ ]:
import math
def convert_numeric_column_to_decimal_with_logarithm(old_df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    new_df = old_df.copy()
    new_df[column_name] = new_df[column_name].apply(lambda x: math.log10(x+1) if x > 0 else 0)
    new_df = convert_numeric_column_to_decimal(new_df, column_name)
    return new_df

training_dataframe = convert_numeric_column_to_decimal_with_logarithm(training_dataframe, "Fare")
training_dataframe.head(10)

## Linear Regression
### Add a constant value
A linear function needs a constant, this will be needed for the maths so we should add a column full of ones

In [ ]:
training_dataframe["Constant"] = 1
training_dataframe.head(10)

### Prepare initial linear co-efficient values
We want to set each of our parameter values to a random number close to 1. The survived column is not a parameter but our desired result/output so we don't include this.

In [ ]:
input_df = training_dataframe.drop("Survived", axis=1)
linear_parameters = np.random.rand(input_df.shape[1]).tolist()
linear_parameters

### Calculate the linear function of our parameters multiplied by our random Coefficients

In [ ]:
def calculate_linear_result() -> np.array:
    return input_df.apply(lambda row: row.dot(linear_parameters), axis=1).to_numpy()

training_dataframe["Initial Linear Result"] = input_df.apply(lambda row: row.dot(linear_parameters), axis=1)

### Gradient Descent

In [ ]:
def optimize_weights(inputs: [pd.DataFrame], target_variables: np.array, parameters: [float], learning_rate: float=0.01, epochs: int=1000) -> [float]:
    for current_epoch in range(epochs):
        # Predicted values
        predicted_values = inputs.apply(lambda row: row.dot(parameters), axis=1).to_numpy()
    
        # Calculate error
        errors = predicted_values - target_variables
        mean_square_error = (errors ** 2).mean()
    
        if current_epoch % 100 == 0: #Print every 100th value
            print(mean_square_error)
    
        # Calculate gradient
        gradient = np.dot(inputs.to_numpy().T, errors) * 2 / len(target_variables)
    
        # Update parameters
        parameters -= learning_rate * gradient
    # Final parameters
    print(f"Optimized weights: {parameters}")
    print(f"Final error: {mean_square_error}")
    return parameters
    
linear_parameters = optimize_weights(inputs=input_df, target_variables=training_dataframe["Survived"].to_numpy(), parameters=linear_parameters)

## Neural Nets
The calculation above was a linear regression as we only use one set of parameters.
Here we'll use two sets of parameters, apply a RELU (Rectified Linear Unit) function and add them together to give us a loss. A RELU function is non-linear and simply replaces every negative number with a 0.

The RELU is needed as adding together two linear functions just gives us another linear function which doesn't give us any more resolution for our calculation. Combinging each linear layer with a non-linear RELU allows us to keep each linear functions utility increasing our algortihms accuracy.

### Create Matrix of Relu Values

In [ ]:
np.random.seed(42)
parameter_matrix = np.random.rand(2, input_df.shape[1]) - 0.5
known_survival_matrix = training_dataframe["Survived"].to_numpy().reshape(-1,1)
inputs = input_df.to_numpy()

### Relu Gradient Descent (non-linear)

In [ ]:
# Gradient descent
for current_epoch in range(1000):
    # Predicted values
    predicted_value_matrix = np.dot(inputs, parameter_matrix.T)
    relu_value_matrix = np.maximum(predicted_value_matrix, 0)
    
    # Calculate error
    errors = relu_value_matrix - known_survival_matrix
    summed_errors = np.sum(errors, axis=1)
    if current_epoch % 100 == 0: #Print every 100th value
        print(summed_errors.mean())
    
    # Calculate gradient
    gradient = np.dot(inputs.T, summed_errors) * 2 / len(training_dataframe["Survived"].to_numpy())
    
    # Update parameters
    parameter_matrix -= 0.01 * gradient
    nn_params = parameter_matrix.sum(axis=0)

# Final parameters
print(f"Optimized weights: {nn_params}")

## Create Titanic survial predictions

Now we'll use the parameters we've calculated to try and make predictions about the survivors in our validation set.

In [ ]:
serving_df

In [ ]:
def estimate_missing_ages(old_df: pd.DataFrame) -> pd.DataFrame:
    new_df = old_df.copy()
    mean_age = old_df["Age"].mean()
    new_df["Age"].fillna(value=mean_age, inplace=True)
    return new_df

def estimate_missing_fares(old_df: pd.DataFrame) -> pd.DataFrame:
    new_df = old_df.copy()
    new_df["Fare"].fillna(value=0, inplace=True)
    return new_df
    
def prepare_data(old_df: pd.DataFrame) -> pd.DataFrame:
    new_df = old_df.copy()
    new_df = remove_irrelevant_data(new_df)
    new_df = estimate_missing_ages(new_df)
    new_df = estimate_missing_fares(new_df)
    print("Searching for NA values:")
    print(new_df.isna().any())
    new_df = convert_ticket_class_to_binary_values(new_df)
    new_df = convert_embarkation_port_to_binary_values(new_df)
    new_df = convert_sex_to_binary_value(new_df)
    new_df = convert_numeric_column_to_decimal(new_df, "Age")
    new_df = convert_numeric_column_to_decimal_with_logarithm(new_df, "Fare")
    new_df["Constant"] = 1
    return new_df
    
serving_df = prepare_data(serving_df)
assert (input_df.columns == serving_df.columns).all()
serving_df

In [ ]:
def create_predictions(validation_df: pd.DataFrame, optimized_weights: np.array) -> np.array:
    return np.dot(validation_df.to_numpy(), optimized_weights)

serving_df["Survival Prediction"] = create_predictions(serving_df, nn_params)
serving_df

## Prepare Submission CSV

In [ ]:
original_validation_df = pd.read_csv(data_path + "test.csv")
submission_df = pd.DataFrame()
submission_df["PassengerId"] = original_validation_df["PassengerId"]
submission_df["Survived"] = serving_df["Survival Prediction"].apply(lambda x: 0 if x < 0.5 else 1)
submission_df.to_csv("submission.csv", index=False)
submission_df